<a href="https://colab.research.google.com/github/LabSWPP12023S2G2/TPInicial/blob/main/datarefUNC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [21]:
url = 'https://raw.githubusercontent.com/LabSWPP12023S2G2/TPInicial/main/datasetUNC.csv'
df = pd.read_csv(url, delimiter=';')

In [22]:
df.columns

Index(['SUB PERIODS', 'EDUCATION', 'PROVINCE', 'SEX', 'AGE',
       'MENTAL DISORDER HISTORY', 'SUIC ATTEMPT HISTORY',
       'LIVING WITH SOMEBODY', 'ECONOMIC INCOME', 'DEPRESSION', 'SUIC RISK',
       'ANXIETY STATE', 'ANXIETY TRAIT'],
      dtype='object')

In [16]:
df = df.dropna(axis=0)

In [17]:
y = df['SUIC RISK']

In [18]:
dk = df.groupby('PROVINCE')

In [19]:
cordoba = dk.get_group('Córdoba')

In [20]:
cordoba

,SUB PERIODS,EDUCATION,PROVINCE,SEX,AGE,MENTAL DISORDER HISTORY,SUIC ATTEMPT HISTORY,LIVING WITH SOMEBODY,ECONOMIC INCOME,DEPRESSION,SUIC RISK,ANXIETY STATE,ANXIETY TRAIT
5,1. EXT POST,Completed postgraduate,Córdoba,woman,35,no,no,yes,yes,9,4,25,12
6,1. EXT POST,Incomplete tertiary or university,Córdoba,woman,21,no,no,yes,yes,5,14,20,22
8,1. EXT POST,Incomplete postgraduate,Córdoba,woman,28,yes,no,yes,no,20,26,28,26
15,1. EXT POST,Incomplete tertiary or university,Córdoba,man,34,no,no,no,yes,5,18,16,11
26,1. EXT POST,Completed postgraduate,Córdoba,man,39,no,no,yes,yes,10,19,37,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,4. EXT POST,Completed postgraduate,Córdoba,woman,32,no,no,yes,yes,6,9,11,10
1092,4. EXT POST,Completed postgraduate,Córdoba,man,76,no,no,yes,yes,3,12,7,15
1095,4. EXT POST,Completed postgraduate,Córdoba,woman,28,no,yes,yes,yes,41,80,51,47
1096,4. EXT POST,Completed tertiary or university,Córdoba,man,39,no,no,yes,yes,10,19,31,25


In [12]:
df_features = ['DEPRESSION', 'ANXIETY STATE']

In [ ]:
x = df[df_features]

In [ ]:
x.describe()

,DEPRESSION,ANXIETY STATE
count,1100.000000,1100.000000
mean,15.695455,31.775455
std,11.101032,14.473519
min,0.000000,1.000000
25%,8.000000,21.000000
50%,13.000000,31.000000
75%,22.000000,42.000000
max,60.000000,66.000000


In [ ]:
x.head()

,DEPRESSION,ANXIETY STATE
0,21,54
1,26,34
2,8,33
3,27,42
4,1,11


In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
df_model = DecisionTreeRegressor(random_state=1)

In [ ]:
df_model.fit(x, y)

DecisionTreeRegressor(random_state=1)

In [ ]:
print("Informar el riesgo de suicidio de las siguientes personas")
print(x.head())
print("El riesgo de suicidio es:")
print(df_model.predict(x.head()))

Informar el riesgo de suicidio de las siguientes personas
   DEPRESSION  ANXIETY STATE
0          21             54
1          26             34
2           8             33
3          27             42
4           1             11
El riesgo de suicidio es:
[37. 46. 27. 70. 19.]


In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
pred_suicrisk = df_model.predict(x)

In [ ]:
mean_absolute_error(y, pred_suicrisk)

3.245757575757576

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_x, val_x, train_y, val_y = train_test_split(x, y, random_state = 0)

In [ ]:
df_model = DecisionTreeRegressor()

In [ ]:
df_model.fit(train_x, train_y)

DecisionTreeRegressor()

In [ ]:
val_pred = df_model.predict(val_x)

In [ ]:
print(mean_absolute_error(val_y, val_pred))

10.65139393939394
